In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np
import time
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

rootPath = "/content/drive/MyDrive/dataset/veremi"
veremiPath = "/veremi"
t2Path = "/veremiWithT2"
csvt2Path = "/simulationscsv"

os.chdir(rootPath+t2Path+csvt2Path)

len(os.listdir())

225

In [3]:
# preprocessing
mid = '/work/ul/ul_vertsys/ul_wqy57'
end = '/veins-maat/simulations/securecomm2018/results'
os.chdir(rootPath+veremiPath)
simulations = pd.Series(os.listdir()).sort_values().reset_index(drop=True)
totalsim = len(simulations)
os.chdir(rootPath+t2Path+csvt2Path)
# cria uma lista com todas simulações para execução parcial
if os.listdir():
  clist = []
  concluded = pd.Series(os.listdir()).sort_values().reset_index(drop=True)
  for concl in concluded.values:
    c = concl.split('.')
    clist.append(int(c[0][-3:]))
else:
  clist = []
tlist = list(set(range(totalsim)))
remainlist = list(set(tlist) - set(clist))
while remainlist:
  os.chdir(rootPath+t2Path+csvt2Path)
  concluded = pd.Series(os.listdir()).sort_values().reset_index(drop=True)
  clist = []
  for concl in concluded.values:
    c = concl.split('.')
    clist.append(int(c[0][-3:]))
  remainlist = list(set(tlist) - set(clist))
  # seleciona o item a ser processado
  if remainlist:
    item = 0
    sim = remainlist[item]
    szremainlist = len(remainlist)
    # start preprocess
    start_time = time.time()
    sims = simulations[sim]
    os.chdir(rootPath+veremiPath+'/'+sims+mid+'/'+sims+end)
    files = pd.Series(os.listdir()).sort_values().reset_index(drop=True)
    files = files.drop(index=files.loc[files.str.contains('.json') == False].index).reset_index(drop=True)

    # ground truth
    dataGT = []
    with open(files[0]) as f:
      for line in f:
        try:
          dataGT.append(json.loads(line.strip()))
        except ValueError:
          pass

    dataGT = pd.DataFrame(dataGT)
    # display(dataGT)

    # retorna todas linhas daquela simulação
    dataLog = []
    lfiles = len(files[1:])
    for id, i in enumerate(files[1:], start=1):
      print('\r',
            str(id)+'º de '+str(lfiles)+' | ',
            'reading simulation '+str(sim)+' |',
            " in {:.2f}".format(time.time() - start_time)+' sec',
            ' faltam '+str(szremainlist)+' simulações.',
            sep='', end='', flush=True)
      with open(files[id]) as f:
        for line in f:
          try:
            row = json.loads(line.strip())
            # add a receiver line to form the pair of communication
            row["receiver"] = int(files[id].split("-")[2])
            row["pxSnd"] = row['pos'][0]
            row["pySnd"] = row['pos'][1]
            row["pzSnd"] = row['pos'][2]
            row["sxSnd"] = row['spd'][0]
            row["sySnd"] = row['spd'][1]
            row["szSnd"] = row['spd'][2]
            dataLog.append(row)
          except ValueError:
            pass

    dataLog = pd.DataFrame(dataLog)

    # add the last know receiver's position 
    vehicles = pd.Series(dataLog['receiver'].unique().astype(int)).sort_values().reset_index(drop=True)
    size = len(vehicles)
    # iv = vehicle index
    for iv, v in enumerate(vehicles):
      # if iv <= 0:
        # for each BSM (type 3) do this
        idxtype2 = dataLog.loc[(dataLog['receiver'].astype(int) == v) & (dataLog['type'] == 2)].index
        for tp in dataLog.loc[(dataLog['type'] == 3) & (dataLog['receiver'].astype(int) == v)].index:
          print('\r',
                'simulation '+str(sim)+' |',
                ' vehicle '+str(iv+1)+' of '+str(size),
                " in {:.2f}".format(time.time() - start_time)+' sec |',
                ' faltam '+str(szremainlist)+' simulações.',
                sep='', end='', flush=True)
          poslist = dataLog['pos'].iloc[np.max(idxtype2[idxtype2 < tp])]
          spdlist = dataLog['spd'].iloc[np.max(idxtype2[idxtype2 < tp])]
          dataLog.at[tp, 'pxRcv'] = poslist[0]
          dataLog.at[tp, 'pyRcv'] = poslist[1]
          dataLog.at[tp, 'pzRcv'] = poslist[2]
          dataLog.at[tp, 'sxRcv'] = spdlist[0]
          dataLog.at[tp, 'syRcv'] = spdlist[1]
          dataLog.at[tp, 'szRcv'] = spdlist[2]

    # Drop “Type2” in JSON files
    dataLog = dataLog.drop(dataLog[dataLog.type == 2].index).reset_index(drop=True)

    # Drop pos, spd, type, noise, spd_noise, pos_noise in JSON files
    dataLog = dataLog.drop(columns=['pos', 'spd', 'type', 'noise', 'spd_noise', 'pos_noise']).reset_index(drop=True)

    # Merge with ground truth to add labels
    dataLog = dataLog.merge(dataGT[['messageID', 'attackerType']])

    # display(dataLog)

    os.chdir(rootPath+t2Path+csvt2Path)
    print('\rSaving sim'+f"{remainlist[item]:03}"+'.csv', sep='', end='', flush=True)
    dataLog.to_csv('sim'+f"{remainlist[item]:03}"+'.csv')
    
    # log the time of preprocessing
    os.chdir(rootPath+t2Path)
    simTime = time.time() - start_time
    data = {"simulation": "sim"+f"{remainlist[item]:03}", "time": simTime}
    try:
      timeDf = pd.read_csv('timePreprocessing.csv', index_col=0)
    except FileNotFoundError:
      timeDf = pd.DataFrame({"simulation": [data['simulation']], "time": [data['time']]})
      timeDf.to_csv('timePreprocessing.csv')
    else:
      if (timeDf['simulation'] == data['simulation']).any():
        timeDf.loc[timeDf['simulation'] == data['simulation'], ['time']] = data['time']
        timeDf.to_csv('timePreprocessing.csv')
      else:    
        timeDf = timeDf.append(data, ignore_index=True)
        timeDf.to_csv('timePreprocessing.csv')

    # retira o item da remainlist
    remainlist.pop(item)

os.chdir(rootPath+t2Path)
timeDf = pd.read_csv('timePreprocessing.csv', index_col=0)
print('\rDone in ', "{:.2f}".format(timeDf['time'].sum())+' sec', sep='', end='', flush=True)

Done in 84117.37 sec